In [1]:
from  tiny.package import *
from  tiny.util import *
from  tiny.lda import *
from  tiny.tfidf import *
tmp = get_tfidf()

2018-09-21 23:46:46,821 util_cache_file.py[79] DEBUG fn:get_tfidf, para:[], kw:{}
2018-09-21 23:46:46,823 util_cache_file.py[21] DEBUG try to read cache from file:./cache/get_tfidf_[]_{}.pkl, type:pkl
2018-09-21 23:46:49,495 util_cache_file.py[26] DEBUG Load to <class 'pandas.core.sparse.frame.SparseDataFrame'> with density:0.00030081144947150695 for pkl@./cache/get_tfidf_[]_{}.pkl
2018-09-21 23:46:51,053 util_cache_file.py[35] DEBUG Return 72727 resut from file cache:./cache/get_tfidf_[]_{}.pkl


In [3]:
tmp.sum(axis=1)

KeyboardInterrupt: 

In [2]:


from tiny.group_label import *
gp_col = 'p_type'
top=3
rootdir = './output/start_close/'
list_ = os.listdir(rootdir)  # 列出文件夹下所有的目录与文件
path_list = sorted(list_, reverse=True)
path_list = [os.path.join(rootdir, item) for item in path_list if item.endswith('csv')]

from multiprocessing import Pool as ThreadPool

pool = ThreadPool(processes=4)

process_file = partial(summary_top_for_individual_file, gp_col=gp_col, top=top)
results = pool.map(process_file, path_list[:3])

results = [item for item in results if len(item)>0]


2018-09-21 23:38:07,796 util_log.py[33] INFO Begin to run summary_top_for_individual_file with:['./output/start_close/deviceid_package_start_close_40_36_33897608_34837990.csv'], {'gp_col': 'p_type', 'top': 3}
2018-09-21 23:38:07,796 util_log.py[33] INFO Begin to run summary_top_for_individual_file with:['./output/start_close/deviceid_package_start_close_40_37_34837990_35780089.csv'], {'gp_col': 'p_type', 'top': 3}
2018-09-21 23:38:07,796 util_log.py[33] INFO Begin to run summary_top_for_individual_file with:['./output/start_close/deviceid_package_start_close_40_38_35780089_36720940.csv'], {'gp_col': 'p_type', 'top': 3}
2018-09-21 23:38:07,802 util_cache_file.py[79] DEBUG fn:cal_duration_for_partition, para:['deviceid_package_start_close_40_37_34837990_35780089.csv'], kw:{}
2018-09-21 23:38:07,802 util_cache_file.py[79] DEBUG fn:cal_duration_for_partition, para:['deviceid_package_start_close_40_38_35780089_36720940.csv'], kw:{}
2018-09-21 23:38:07,804 util_cache_file.py[79] DEBUG fn:cal

In [7]:
for item in results:
    print(item.shape)

xx = pd.concat(results)

print(len(list(xx.columns)))

(1827, 102)
(1927, 102)
(1924, 102)
102


In [ ]:

#group by specific label
#p_type, p_sub_type, combine_type, package
gp_col = 'p_type' 
top = 3
label_list = []
value = 'package_cnt_daily'#,'package_dur_daily', 'duration_sum'

#group by top#n
gp = ex_input.groupby(['device', gp_col]).agg({'start':['count'], gp_col:['nunique'], 'duration':['sum'], 'start_base':['nunique']})
gp.columns = ["_".join(x) for x in gp.columns.ravel()]

gp[f'{gp_col}_cnt_daily'] = gp[f'start_count'] / gp['start_base_nunique']
gp[f'{gp_col}_dur_daily'] = gp[f'duration_sum'] / gp['start_base_nunique']

gp = gp.reset_index()
gp = gp.sort_values(['device',f'{gp_col}_cnt_daily'], ascending=False)
gp['cum_sn'] = gp.groupby('device').cumcount()
gp = gp[gp.cum_sn < top]

gp
gp00 = gp.pivot(index='device', columns='cum_sn', values=f'{gp_col}_cnt_daily')
gp00.columns = [f'{gp_col}_cnt_top#{item}' for item in gp00.columns]

gp01 = gp.pivot(index='device', columns='cum_sn', values=f'{gp_col}_dur_daily')
gp01.columns = [f'{gp_col}_dur_top#{item}' for item in gp01.columns]

gp02 = gp.pivot(index='device', columns='cum_sn', values=gp_col)
gp02.columns = [f'{gp_col}_name_top#{item}' for item in gp02.columns]





gp = ex_input.groupby(['device', gp_col]).agg({'start':['count'],  'duration':['sum'], 'start_base':['nunique']})
gp.columns = ["_".join(x) for x in gp.columns.ravel()]

gp[f'{gp_col}_count'] = gp.count(axis=1)
gp[f'{gp_col}_cnt_daily'] = gp[f'start_count'] / gp['start_base_nunique']
gp[f'{gp_col}_dur_daily'] = gp[f'duration_sum'] / gp['start_base_nunique']
gp
gp = gp.reset_index()
gp = gp.sort_values(['device',f'{gp_col}_cnt_daily'], ascending=False)
gp
gp['cum_sn'] = gp.groupby('device').cumcount()
gp11 = gp.pivot(index='device', columns=gp_col, values=f'{gp_col}_cnt_daily')
gp12 = gp.pivot(index='device', columns=gp_col, values=f'{gp_col}_dur_daily')



tmp = ex_input.groupby('device')[gp_col].nunique().to_frame()
tmp.rename(columns={gp_col:f'{gp_col}_count'}, inplace=True)
 

df = pd.concat([gp00, gp01, gp02, gp11, gp12, tmp], axis=1)

df.fillna(0)
 
# gp[[f'{gp_col}_count','cum_sn']] = gp.groupby('device').agg({'duration_sum':'count', 'package_nunique':'cumcount'})
# gp
#len(list(gp.columns))

In [ ]:
tmp = get_package_label()
len(tmp.p_type.value_counts())

In [ ]:
%matplotlib inline
pd.set_option('display.height', 1000)
df['month'] = df.start_base.dt.to_period('M')
tmp = df.groupby('month')['month'].count()
tmp = tmp.to_frame()
tmp.rename(columns={'month':'count'}, inplace=True)
tmp.reset_index(inplace=True)
#tmp.dtypes

#tmp.plot.line()

In [ ]:
#tmp.set_index('month', inplace=True)
tmp

In [ ]:
total=df.groupby(['device']).agg({'package':'nunique', 'duration':'sum'})


gp = df.groupby(['device', 'weekday']).agg({'package':'nunique', 'day_duration':'sum'})
gp
gp.reset_index(inplace=True)
gp
gp1=gp.pivot(index='device', columns='weekday',values='package')
gp1.columns = [f'package_{col}' for col in gp1.columns]

gp2=gp.pivot(index='device', columns='weekday',values='day_duration')
gp2.columns = [f'duration_{col}' for col in gp2.columns]

gp1.head()

In [ ]:
merge = pd.concat([gp1, gp2, wk, total], axis=1)
merge.head()

In [ ]:


#转换为Percentage
type_= 'package'
for col in [col for col in merge.columns if f'{type_}_' in col]:
    #print(col)
    merge[col] = merge[col]/merge[type_]
    
type_= 'duration'
for col in [col for col in merge.columns if f'{type_}_' in col]:
   # print(col)
    merge[col] = merge[col]/merge[type_]
    
merge.head()

In [ ]:
df.describe()

In [ ]:
for col in tmp.columns:
    if 'Unknown' in col:
        print(col)
 


In [ ]:
pkg_label = get_package_label()
pkg_label.set_index('package', inplace=True)
pkg_label

In [ ]:
extend_time_span(version=0, trunc_long_time=False, mini=True, groupby=['device'], prefix='tol')


In [ ]:
columns = [ col for col in install.columns ]
print(columns[:4])

In [ ]:
vectorizer = CountVectorizer()
cntTf_app = vectorizer.fit_transform(apps)

str(cntTf_app[:3])
# cntTf_app = pd.DataFrame(cntTf_app,
#                          columns=vectorizer.get_feature_names(),
#                          index=deviceid_packages.device)

In [ ]:
mini = get_start_closed()
tmp[tmp.device.isin(mini.device)][['device','duration_max_min', 'tol_day_cnt_nunique', 'gap']].sort_values('gap', ascending=False)


#f993126b72f6297440c2dc8534b20a0b  49
#fbf985012f537449b455d6a554479f8b  1161




In [ ]:
from  tiny.lda import *
get_lda_from_usage()

In [ ]:
device= 'fa68862f9484c62aca5c3a4b8a2c70bc'
mini[mini.device==device].sort_values('start')

In [ ]:
tmp[tmp.device==device]

In [ ]:
df2

In [ ]:
#df2.set_index('device', inplace=True)
df.sort_index(ascending=False)


In [ ]:
split.head()

In [ ]:
print(pd.to_datetime('1488332491588', unit='ms'))  #fc6383d2bdb463fd01db6731594a6d36	07e967d75aab2f6a52c558695a572a7c
print(pd.to_datetime('1488332506604', unit='ms'))
print(pd.to_datetime('1488363700875', unit='ms'))
print(pd.to_datetime('1488363707487', unit='ms'))

In [ ]:
tmp = extend_time(tmp,2)
#tmp['start'].astype
#tmp['start'] = pd.to_datetime(tmp['start'])
#tmp['start'].dt.date

In [ ]:

# def extend_percent(df):
#     total = get_percent_duration(df, ['device'], 'total')

#     max_week = get_max_week(df)

#     merge = df.merge(max_week, on=['device', 'weekbegin'])

#     max = get_percent_duration(merge, ['device'], prefix='max')

#     return pd.concat( [total, max], axis=1 )


# total = get_percent_duration(df, ['device'], 'total')


# df = tmp
# max_week = get_max_week(df)
# max_week.head()
# merge = df.merge(max_week, on=['device', 'weekbegin'])

 
# def get_percent_duration(df, groupby=['device', 'weekday'], prefix=None):
#     prefix = groupby[-1] if prefix is None else prefix
#     columns = [key for key in df.columns if 'span_' in key]
#     gp_map = [(key, 'sum') for key in columns if 'span_' in key]
#     gp_map = dict(gp_map)
#     # print(gp_map)

#     gp_map['package'] = 'nunique'
#     gp_map['start_base'] = 'nunique'
#     df = df.groupby(groupby).agg(gp_map)
#     #     df = df.groupby(groupby).agg({'span_0':'sum','span_1':'sum','span_2':'sum',
#     #                                                  'span_3':'sum','package':'nunique' })
#     df['total'] = df[[key for key in columns if 'span_' in key]].sum(axis=1)

#     for col in columns:
#         df[f'{col}_p'] = round(df[col] / df['total'], 3)

#     df.rename({'package':'package_cnt', 'start_base':'start_base_cnt'}, axis=1, inplace=True)

#     df.columns = [f'{prefix}_{key}' for key in df.columns]
#     return df

per = extend_percent(tmp)
per.head()

# max_week = get_max_week(tmp)
# max_week

# merge= tmp.merge(max_week, on = ['device','weekbegin'])

# print(len(merge))

# get_percent_duration(merge, ['device'], prefix='max')


    

In [ ]:
per
#tmp[tmp.device=='fc6383d2bdb463fd01db6731594a6d36'].sort_values('start')

In [ ]:
print('yes')

In [ ]:
#tmp['start_base']  = tmp['start'].dt.date
# tmp = tmp[tmp.device.isin(['fff885dafce62310951c835247f30a38',
# 'fc65f1731b785a8ce82d20a30dee9259'])]

df = tmp.groupby(['device','weekbegin']).package.nunique().to_frame().reset_index()

# .to_frame().reset_index()
# df.sort_values(['device', 'package'])

df.sort_values(by=['device', 'package', 'weekbegin'], ascending=False).groupby('device').nth(0)


In [ ]:
xx = df



In [ ]:
from lightgbm.sklearn import LGBMRegressor, LGBMClassifier
from xgboost.sklearn import XGBRegressor
import lightgbm as lgb
from sklearn.cross_validation import train_test_split

from tiny.lda import *
from  tiny.util import *

# New add
# deviceid_train.rename({'device_id':'device'}, axis=1, inplace=True)
deviceid_train = get_lda_feature()
deviceid_train2 = get_lda_from_usage(mini=mini)

core_list = ['0', '1', '2', '3','4']
# for col in core_list:
#     deviceid_train_2[col] =  deviceid_train_2[col].apply(lambda val: 1 if val > 0 else 0)
deviceid_train = pd.concat([deviceid_train,deviceid_train2[core_list] ], axis=1)

deviceid_train = extend_feature(span_no=24, input=deviceid_train, trunc_long_time=False)

#deviceid_train = extend_feature(span_no=4,input=deviceid_train,  trunc_long_time=False)



#print(len(deviceid_train))
#deviceid_train.groupby('max_day_cnt')['max_day_cnt'].count()

#
#
# col_drop = [item for item in deviceid_train.columns if 'max_' in str(item)]
# deviceid_train.drop(columns=col_drop, inplace=True )

#deviceid_train.drop(columns=['tfidf_sum'], inplace=True )
deviceid_train.head()


train=deviceid_train[deviceid_train['sex'].notnull()]
test=deviceid_train[deviceid_train['sex'].isnull()]

X = train.drop(['sex', 'age', 'sex_age', 'device'], axis=1)
Y = train['sex_age']
Y_CAT = pd.Categorical(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y_CAT.labels, test_size=0.3, random_state=666)
# lgb_train = lgb.Dataset(X_train, label=y_train)
# lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'max_depth': 3,
    'metric': {'multi_logloss'},
    'num_class': 22,
    'objective': 'multiclass',
    'random_state': 47,

    "min_data_in_leaf":1000,
    'verbose': -1

}


gbm = LGBMClassifier(
        n_estimators=1000,

        )

gbm.set_params(**params)

print(gbm)

gbm.fit(X_train, y_train, eval_set=[(X_test, y_test)],early_stopping_rounds=100,)

#y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)#
print('Feature importances:', list(gbm.feature_importances_))

best = round(gbm.best_score_.get('valid_0').get('multi_logloss'), 5)
best


pre_x=test.drop(['sex','age','sex_age','device'],axis=1)


In [ ]:
sub=pd.DataFrame(gbm.predict_proba(pre_x.values,num_iteration=gbm.best_iteration_))

sub.head()

In [ ]:
tmp = extend_cols(tmp)
tmp, tmp_new = split_days(tmp)

tmp.head()

In [ ]:
tmp, tmp_new = split_days(tmp)
tmp.head()

In [ ]:
trainn_label = get_train_label()
trainn_label.head()

In [ ]:
from  tiny.util import *

tmp = extend_feature( span_no=2, trunc_long_time=False, mini=True)

In [ ]:
tmp.head()

In [ ]:
tmp = extend_time_span(version=1, trunc_long_time=False, mini=True, groupby=['device'], prefix='tol')
tmp.head()

In [ ]:
from tiny.lda import *
from  tiny.util import *

tmp = extend_package(version=1)
tmp.head(3).sum(axis=1)

In [ ]:
tmp = tmp/tmp

tmp.head(3).sum(axis=1)

In [ ]:
tmp.head(3).count(axis=1)

In [ ]:
tmp.head(3)